In [ ]:
!pip install keras-nightly
!pip install --quiet tensorflow-dtensor
!pip install --quiet --upgrade tensorflow-datasets

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.experimental import dtensor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tensorflow-dtensor (from versions: none)
ERROR: No matching distribution found for tensorflow-dtensor


In [ ]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(
        phy_devices[0],
        [tf.config.LogicalDeviceConfiguration()] * ncpu)

configure_virtual_cpus(8)
tf.config.list_logical_devices('CPU')

devices = [f'CPU:{i}' for i in range(8)]

In [ ]:
try:
    tf.keras.backend.experimental.enable_tf_random_generator()
except AttributeError:
    try:
        tf.keras.backend.enable_tf_random_generator()
    except AttributeError:
        try:
          tf.random.set_seed(1337)
        except AttributeError:
          print("Warning: enable_tf_random_generator not found. Randomness might not be fully controlled.")
          pass

In [ ]:
mesh = dtensor.create_mesh([("batch", 8)], devices=devices)

In [ ]:
example_weight_layout = dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)  # or
example_weight_layout = dtensor.Layout.replicated(mesh, rank=2)

In [ ]:
example_data_layout = dtensor.Layout(['batch', dtensor.UNSHARDED], mesh)  # or
example_data_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)

In [ ]:
unsharded_layout_2d = dtensor.Layout.replicated(mesh, 2)
unsharded_layout_1d = dtensor.Layout.replicated(mesh, 1)

In [ ]:
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
#   tf.keras.layers.Dense(128,
#                         activation='relu',
#                         name='d1',
#                         kernel_initializer=unsharded_layout_2d,
#                         bias_initializer=unsharded_layout_1d),
#   tf.keras.layers.Dense(10,
#                         name='d2',
#                         kernel_initializer=unsharded_layout_2d,
#                         bias_initializer=unsharded_layout_1d)
# ])

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128,
                              activation='relu',
                              name='d1',
                              kernel_initializer=tf.keras.initializers.GlorotUniform(),  # Initialize using Keras initializer directly
                              bias_initializer=tf.keras.initializers.Zeros()),  # Initialize using Keras initializer directly
                              # kernel_layout=unsharded_layout_2d,  # Remove these lines
                              # bias_layout=unsharded_layout_1d), # Remove these lines
        tf.keras.layers.Dense(10,
                              name='d2',
                              kernel_initializer=tf.keras.initializers.GlorotUniform(),  # Initialize using Keras initializer directly
                              bias_initializer=tf.keras.initializers.Zeros())  # Initialize using Keras initializer directly
                              # kernel_layout=unsharded_layout_2d,  # Remove these lines
                              # bias_layout=unsharded_layout_1d)  # Remove these lines
    ])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ d1 (Dense)                           │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ d2 (Dense)                           │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # Creating DTensor variables with desired layouts
# def glorot_uniform_dtensor(shape, dtype=tf.float32, layout=None):
#   """Create a DTensor variable initialized with GlorotUniform."""
#   initializer = tf.keras.initializers.GlorotUniform()
#   # Call the initializer to get a Tensor and then pack it into a DTensor
#   # Important: Convert to DTensor after initialization
#   tensor = initializer(shape, dtype=dtype)
#   with tf.device(layout.mesh.local_devices()[0]):  # Initialize on the first device of the mesh
#     dtensor_var = dtensor.DVariable(tensor, layout=layout)
#   return dtensor_var


# def zeros_dtensor(shape, dtype=tf.float32, layout=None):
#   """Create a DTensor variable initialized with zeros."""
#   initializer = tf.keras.initializers.Zeros()
#   # Call the initializer to get a Tensor and then pack it into a DTensor
#   # Important: Convert to DTensor after initialization
#   tensor = initializer(shape, dtype=dtype)
#   with tf.device(layout.mesh.local_devices()[0]):  # Initialize on the first device of the mesh
#     dtensor_var = dtensor.DVariable(tensor, layout=layout)
#   return dtensor_var


# # Defining the model with DTensor variables
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128,
#                           activation='relu',
#                           name='d1',
#                           kernel_initializer=lambda shape, dtype: glorot_uniform_dtensor(shape, dtype, layout=unsharded_layout_2d),
#                           bias_initializer=lambda shape, dtype: zeros_dtensor(shape, dtype, layout=unsharded_layout_1d)),
#     tf.keras.layers.Dense(10,
#                           name='d2',
#                           kernel_initializer=lambda shape, dtype: glorot_uniform_dtensor(shape, dtype, layout=unsharded_layout_2d),
#                           bias_initializer=lambda shape, dtype: zeros_dtensor(shape, dtype, layout=unsharded_layout_1d))
# ])

# # Accessing DTensor layouts
# for weight in model.weights:
#   # Weights are now DTensors, so they have the 'layout' attribute
#   print(f'Weight name: {weight.name} with layout: {weight.layout}')
#   break

In [ ]:
# for weight in model.weights:
#   print(f'Weight name: {weight.name} with layout: {weight.layout}')
#   break

In [ ]:
for layer in model.layers:
    for weight in layer.weights:
        print(f"Layer: {layer.name}, Weight shape: {weight.shape}, dtype: {weight.dtype}")

Layer: d1, Weight shape: (784, 128), dtype: float32
Layer: d1, Weight shape: (128,), dtype: float32
Layer: d2, Weight shape: (128, 10), dtype: float32
Layer: d2, Weight shape: (10,), dtype: float32


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.AVJ9VS_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.AVJ9VS_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [ ]:
batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
@tf.function
def train_step(model, x, y, optimizer, metrics):
  with tf.GradientTape() as tape:
    logits = model(x, training=True)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(
        y, logits, from_logits=True))

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  for metric in metrics.values():
    metric.update_state(y_true=y, y_pred=logits)

  loss_per_sample = loss / len(x)
  results = {'loss': loss_per_sample}
  return results

In [ ]:
@tf.function
def eval_step(model, x, y, metrics):
  logits = model(x, training=False)
  loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(
        y, logits, from_logits=True))

  for metric in metrics.values():
    metric.update_state(y_true=y, y_pred=logits)

  loss_per_sample = loss / len(x)
  results = {'eval_loss': loss_per_sample}
  return results

In [ ]:
def pack_dtensor_inputs(images, labels, image_layout, label_layout):
  num_local_devices = image_layout.mesh.num_local_devices()
  images = tf.split(images, num_local_devices)
  labels = tf.split(labels, num_local_devices)
  images = dtensor.pack(images, image_layout)
  labels = dtensor.pack(labels, label_layout)
  return  images, labels

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)#, mesh=mesh)
metrics = {'accuracy': tf.keras.metrics.SparseCategoricalAccuracy()}#mesh=mesh)}
eval_metrics = {'eval_accuracy': tf.keras.metrics.SparseCategoricalAccuracy()}#mesh=mesh)}

In [ ]:
num_epochs = 3

image_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=4)
label_layout = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

for epoch in range(num_epochs):
  print("============================")
  print("Epoch: ", epoch)
  for metric in metrics.values():
    metric.reset_state()
  step = 0
  results = {}
  pbar = tf.keras.utils.Progbar(target=None, stateful_metrics=[])
  for input in ds_train:
    images, labels = input[0], input[1]
    images, labels = pack_dtensor_inputs(
        images, labels, image_layout, label_layout)

    results.update(train_step(model, images, labels, optimizer, metrics))
    for metric_name, metric in metrics.items():
      results[metric_name] = metric.result()

    pbar.update(step, values=results.items(), finalize=False)
    step += 1
  pbar.update(step, values=results.items(), finalize=True)

  for metric in eval_metrics.values():
    metric.reset_state()
  for input in ds_test:
    images, labels = input[0], input[1]
    images, labels = pack_dtensor_inputs(
        images, labels, image_layout, label_layout)
    results.update(eval_step(model, images, labels, eval_metrics))

  for metric_name, metric in eval_metrics.items():
    results[metric_name] = metric.result()

  for metric_name, metric in results.items():
    print(f"{metric_name}: {metric.numpy()}")

Epoch:  0


/usr/local/lib/python3.11/dist-packages/tensorflow/dtensor/python/layout.py:242: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  return hash(self.as_proto().SerializeToString(deterministic=True))


    469/Unknown 41s 59ms/step - loss: 0.2352 - accuracy: 0.1756
loss: 0.13356508314609528
accuracy: 0.2025899738073349
eval_loss: 0.030852090567350388
eval_accuracy: 0.16138985753059387
Epoch:  1
    469/Unknown 41s 87ms/step - loss: 0.1189 - accuracy: 0.2135
loss: 0.01667550951242447
accuracy: 0.2271651178598404
eval_loss: 0.036255061626434326
eval_accuracy: 0.19099746644496918
Epoch:  2
    469/Unknown 20s 43ms/step - loss: 0.0896 - accuracy: 0.2346
loss: 0.09035062044858932
accuracy: 0.23676322400569916
eval_loss: 0.1539943963289261
eval_accuracy: 0.1770680844783783


In [ ]:
class SubclassedModel(tf.keras.Model):

  def __init__(self, name=None):
    super().__init__(name=name)
    self.feature = tf.keras.layers.Dense(16)
    self.feature_2 = tf.keras.layers.Dense(24)
    self.dropout = tf.keras.layers.Dropout(0.1)

  def call(self, inputs, training=None):
    x = self.feature(inputs)
    x = self.dropout(x, training=training)
    return self.feature_2(x)

In [ ]:
layout_map = tf.keras.dtensor.experimental.LayoutMap(mesh=mesh)

layout_map['feature.*kernel'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)
layout_map['feature.*bias'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

with layout_map.scope():
  subclassed_model = SubclassedModel()

AttributeError: module 'keras._tf_keras.keras' has no attribute 'dtensor'

In [ ]:
dtensor_input = dtensor.copy_to_mesh(tf.zeros((16, 16)), layout=unsharded_layout_2d)
# Trigger the weights creation for subclass model
subclassed_model(dtensor_input)

print(subclassed_model.feature.kernel.layout)

In [ ]:
layout_map = tf.keras.dtensor.experimental.LayoutMap(mesh=mesh)

layout_map['feature.*kernel'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=2)
layout_map['feature.*bias'] = dtensor.Layout.batch_sharded(mesh, 'batch', rank=1)

In [ ]:
with layout_map.scope():
  inputs = tf.keras.Input((16,), batch_size=16)
  x = tf.keras.layers.Dense(16, name='feature')(inputs)
  x = tf.keras.layers.Dropout(0.1)(x)
  output = tf.keras.layers.Dense(32, name='feature_2')(x)
  model = tf.keras.Model(inputs, output)

print(model.layers[1].kernel.layout)

In [ ]:
with layout_map.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(16, name='feature', input_shape=(16,)),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(32, name='feature_2')
  ])

print(model.layers[2].kernel.layout)